In [1]:
import os
import cv2
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
import os
import cv2
import numpy as np

X = []  # изображения
y = []  # метки (0 - подделка, 1 - настоящая подпись)

base_path = 'signatures'  # Путь к папке (не ZIP!)

for person_folder in os.listdir(base_path):
    person_path = os.path.join(base_path, person_folder)

    # Пропускаем, если это не папка
    if not os.path.isdir(person_path):
        continue

    for filename in os.listdir(person_path):
        file_path = os.path.join(person_path, filename)

        # Пропускаем скрытые/неизвестные файлы
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        # Проверка, что изображение загружено
        if img is None:
            print(f"Ошибка загрузки: {file_path}")
            continue

        # Обработка изображения
        img = cv2.resize(img, (220, 155))  # стандартный размер
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        X.append(img)

        # ИСПРАВЛЕНИЕ: Определение метки на основе папки и имени файла
        if person_folder == 'full_org' or 'original' in filename.lower():
            y.append(1)  # настоящая подпись
        else:
            y.append(0)  # поддельная подпись

X = np.array(X)
y = np.array(y)

# Проверим распределение меток
print(f"Общее количество изображений: {len(X)}")
print(f"Настоящие подписи (1): {np.sum(y == 1)}")
print(f"Поддельные подписи (0): {np.sum(y == 0)}")
print(f"Процент настоящих подписей: {np.mean(y == 1)*100:.1f}%")

Общее количество изображений: 2640
Настоящие подписи (1): 1320
Поддельные подписи (0): 1320
Процент настоящих подписей: 50.0%


In [5]:
# Нормализация пикселей (0-1)
X = X / 255.0

# Добавляем канал (для чёрно-белых изображений)
X = X.reshape(-1, 155, 220, 1)

# Разделение на обучение и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(155, 220, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 выходной нейрон для задачи бинарной классификации
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 31s 413ms/step - accuracy: 0.6946 - loss: 0.5849 - val_accuracy: 0.7652 - val_loss: 0.5082
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 351ms/step - accuracy: 0.8338 - loss: 0.3775 - val_accuracy: 0.7860 - val_loss: 0.4651
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 353ms/step - accuracy: 0.9209 - loss: 0.2002 - val_accuracy: 0.7992 - val_loss: 0.4873
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 349ms/step - accuracy: 0.9692 - loss: 0.0971 - val_accuracy: 0.8163 - val_loss: 0.5645
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 351ms/step - accuracy: 0.9905 - loss: 0.0463 - val_accuracy: 0.8277 - val_loss: 0.6891
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 350ms/step - accuracy: 0.9867 - loss: 0.0417 - val_accuracy: 0.8220 - val_loss: 0.6272
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 353ms/step - accuracy: 0.9929 - loss: 0.0247 - val_accuracy: 0.8182 - val_loss: 0.7206
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 25s 376ms/step - accuracy: 0.9938 - loss: 0.0177 - val_accu

In [6]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.2f}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.7917 - loss: 0.7377
Test accuracy: 0.79


In [15]:
def predict_signature(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (220, 155))
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    img = img / 255.0
    img = img.reshape(1, 155, 220, 1)

    prediction = model.predict(img)

    if prediction >= 0.5:
        print(f"Prediction: Genuine Signature ({prediction[0][0]:.2f})")
    else:
        print(f"Prediction: Forged Signature ({prediction[0][0]:.2f})")

In [8]:
# Сохраняем модель
model.save('signature_verification_model.h5')

In [16]:
from tensorflow.keras.models import load_model

# Загружаем сохранённую модель
model = load_model('signature_verification_model.h5')

# Теперь модель готова к использованию!
predict_signature('example/norm_1_1.png')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Prediction: Genuine Signature (0.92)


In [8]:
from tensorflow.keras.models import load_model

In [ ]:
# Загружаем сохранённую модель
model = load_model('signature_verification_model.h5')

In [10]:
import tensorflow as tf

# Загружаем модель из .h5
model = tf.keras.models.load_model("signature_verification_model.h5")

# Конвертация в .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (опционально) включаем квантизацию
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Сохраняем
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ovtti\AppData\Local\Temp\tmpwq1zsv3w\assets


INFO:tensorflow:Assets written to: C:\Users\ovtti\AppData\Local\Temp\tmpwq1zsv3w\assets


Saved artifact at 'C:\Users\ovtti\AppData\Local\Temp\tmpwq1zsv3w'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 155, 220, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2378516184912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492003152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492007184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492007568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492007952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492003920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492006416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2378492006608: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Загружаем сохранённую модель
model = load_model('signature_verification_model.h5')

# Переобучаем модель на новых данных
history = model.fit(
    X_new,  # новые изображения
    y_new,  # новые метки
    epochs=5,  # количество новых эпох
    batch_size=32,
    validation_data=(X_test, y_test)  # для проверки на старых данных
)


In [ ]:
# ПЕРЕОБУЧЕНИЕ МОДЕЛИ С ИСПРАВЛЕННЫМИ ДАННЫМИ
print("Переобучение модели с исправленными метками...")

# Создаем новую модель
model_fixed = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(155, 220, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 выходной нейрон для задачи бинарной классификации
])

model_fixed.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Обучаем модель с исправленными данными
history_fixed = model_fixed.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Оцениваем модель
test_loss, test_acc = model_fixed.evaluate(X_test, y_test)
print(f"Test accuracy с исправленными данными: {test_acc:.2f}")

# Сохраняем исправленную модель
model_fixed.save('signature_verification_model_fixed.h5')


In [ ]:
# ТЕСТИРОВАНИЕ ИСПРАВЛЕННОЙ МОДЕЛИ
def predict_signature_fixed(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (220, 155))
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    img = img / 255.0
    img = img.reshape(1, 155, 220, 1)

    prediction = model_fixed.predict(img)

    if prediction >= 0.5:
        print(f"Prediction: Genuine Signature ({prediction[0][0]:.2f})")
    else:
        print(f"Prediction: Forged Signature ({prediction[0][0]:.2f})")

# Тестируем на примерах
print("Тестирование исправленной модели:")
print("\n1. Тест на настоящей подписи (original_1_1.png):")
predict_signature_fixed('example/original_1_1.png')

print("\n2. Тест на поддельной подписи (forgeries_1_1.png):")
predict_signature_fixed('example/forgeries_1_1.png')
